**Astroinformatica II (Semester 2 2024)**

# Tutorial Session 3: Parallelization

*N. Hernitschek, 2022*



---
## Contents
* [Parallelization with Multiprocessing](#first-bullet)
* [Avoiding slow program/ data structures](#second-bullet)
* [Parallelization with Multithreading](#third-bullet)
* [Summary](#fourth-bullet)


## 1. Parallelization with Multiprocessing<a class="anchor" id="first-bullet"></a>

When processing large data sets, parallelization is usually possible and thus recommended. In most cases, such code runs on a large number of data independently from each other.
e.g.: classifying objects by their light curves, calculating features for objects, estimating the distances...

`multiprocessing` is a package that supports spawning processes using an API similar to the threading module. It runs on both Unix (including Mac) and Windows.

More on this: https://docs.python.org/3/library/multiprocessing.html



**Example:** a template to process multiple light curves in parallel

In [2]:
from multiprocessing import Pool

# demo code for processing one light curve
def one_lc(param):

    i, filename = param
    
    # here goes the code processing something, i.e.: open the file, carry out computations on light curve data
    
    return filename

pool = Pool(processes=9)

#a list of light curve files
filenames = ['lightcurve_1.lc','lightcurve_2.lc','lightcurve_3.lc']

params = [(i, filename) for i, filename in enumerate(filenames)]

it = pool.imap_unordered(one_lc, params)


for i, res in enumerate(it):

    print('res', res)


pool.terminate()
    
    

res lightcurve_1.lc
res lightcurve_2.lc
res lightcurve_3.lc


**Caution**: Some large **clusters/ supercomputers** prefer to have parallelization implemented in a different way. E.g.: Some clusters have *quotas* on the memory usage, CPU usage, and the runtime of your code within one thread/ process, so it can make sense to rather split up your code in multiple **jobs** instead of using Python's parallelization. When using a cluster/ supercomputer, you will find this in the system's documentation.

## 2. Parallelization with Multithreading<a class="anchor" id="second-bullet"></a>

Python provides a module that enables the construction and administration
of threads, thus making multithreaded applications easier to implement.

In [6]:
import threading

def task():
    print(thread.is_alive())
    print("Thread task executed")
    
# Create a thread
thread = threading.Thread(target=task)
print(thread.is_alive())
# Start the thread
thread.start()
print(thread.is_alive())
# Wait for the thread to complete
thread.join()
print(thread.is_alive())
print("Thread execution completed")

print(thread.is_alive())




False
True
Thread task executed
False
False
Thread execution completed
False


### Thread Synchronization

Programming for many threads requires careful consideration of thread synchronization. Preventing conflicts and race conditions entails coordinating the execution of several threads and ensuring that shared resources are accessed and modified securely. Threads can interfere with one another without adequate synchronization, resulting in data corruption, inconsistent results, or unexpected behavior.

Thread synchronization is necessary when multiple threads access shared resources or variables simultaneously. The primary goals of synchronization are:

#### Mutual Exclusion

Ensuring that only one thread can access a shared resource or a critical code section at a time. This prevents data corruption or inconsistent states caused by concurrent modifications.


#### Coordination

Allowing threads to communicate and coordinate their activities effectively. This includes tasks like signaling other threads when a condition is met or waiting for a certain condition to be satisfied before proceeding.
Synchronization Techniques


Python provides various synchronization mechanisms to address thread synchronization needs. Some commonly used techniques include locks (mutexes), semaphores, and condition variables.






### Locks

A lock, usually called a **mutex**, is a fundamental primitive for synchronization that permits mutual exclusion. While other threads wait for the lock to be released, it ensures that only one thread can ever acquire the lock. For this function, the Python threading library offers a Lock class.


In this example, a shared counter variable is incremented by multiple threads. The Lock object, `counter_lock`, ensures mutual exclusion while accessing and modifying the counter.


In [7]:
import threading

counter = 0
counter_lock = threading.Lock()

def increment():
    global counter
    with counter_lock:
        counter += 1

# Create multiple threads to increment the counter
threads = []
for _ in range(10):
    t = threading.Thread(target=increment)
    threads.append(t)
    t.start()

# Wait for all threads to complete
for t in threads:
    t.join()
print("Counter:", counter)

Counter: 10


### Semaphores

A semaphore is a synchronization object that maintains a count. It allows multiple threads to enter a critical section up to a specified limit. If the limit is reached, subsequent threads will be blocked until a thread releases the semaphore. The `threading` module provides a `Semaphore` class for this purpose.

In this example, a semaphore with a limit of 3 controls access to a shared resource. Only three threads can enter the critical section at a time, while others wait for the semaphore to be released.

In [8]:
import threading

semaphore = threading.Semaphore(3)  # Allow 3 threads at a time
resource = []

def access_resource():
    with semaphore:
        resource.append(threading.current_thread().name)
        
# Create multiple threads to access the resource
threads = []

for i in range(10):
    t = threading.Thread(target=access_resource, name=f"Thread-{i+1}")
    threads.append(t)
    t.start()
    
# Wait for all threads to complete
for t in threads:
    t.join()
    
print("Resource:", resource)

Resource: ['Thread-1', 'Thread-2', 'Thread-3', 'Thread-4', 'Thread-5', 'Thread-6', 'Thread-7', 'Thread-8', 'Thread-9', 'Thread-10']


### Condition Variables

Condition variables allow threads to wait for a specific condition to be met before proceeding. They provide a mechanism for threads to signal each other and coordinate their activities. The `threading` module provides a `Condition` class for this purpose.

In [11]:
import threading

buffer = []
buffer_size = 5
buffer_lock = threading.Lock()
buffer_not_full = threading.Condition(lock=buffer_lock)
buffer_not_empty = threading.Condition(lock=buffer_lock)

def produce_item(item):
    with buffer_not_full:
        while len(buffer) >= buffer_size:
            buffer_not_full.wait()
        buffer.append(item)
        buffer_not_empty.notify()
        
def consume_item():
    with buffer_not_empty:
        while len(buffer) == 0:
            buffer_not_empty.wait()
        item = buffer.pop(0)
        buffer_not_full.notify()
        return item
    
# Create producer and consumer threads
producer = threading.Thread(target=produce_item, args=("Item 1",))
consumer = threading.Thread(target=consume_item)
producer.start()
consumer.start()
producer.join()
consumer.join()

## 3. Concurrency vs. Parallelism<a class="anchor" id="third-bullet"></a>

The following example shows the principle of **concurrency**:


We create five threads and assign each to execute the task function with a different name. Concurrency is enabled via multithreading by enabling numerous threads within a single process.

You'll observe that the tasks start and complete in an interleaved manner, indicating concurrent execution.

In this example, a producer thread produces items and adds them to a shared buffer, while a consumer thread consumes items from the buffer. The condition variables `buffer_not_full` and `buffer_not_empty` synchronize the producer and consumer threads, ensuring that the buffer is not full before producing and not empty before consuming.


In [12]:
import threading
import time

def task(name):
    print(f"Task {name} started")
    time.sleep(2)  # Simulating some time-consuming task
    print(f"Task {name} completed")
    
# Creating multiple threads
threads = []

for i in range(5):
    t = threading.Thread(target=task, args=(i,))
    threads.append(t)
    t.start()
    
# Waiting for all threads to complete
for t in threads:
    t.join()
    
print("All tasks completed")

Task 0 started
Task 1 started
Task 2 started
Task 3 started
Task 4 started
Task 0 completed
Task 1 completed
Task 2 completed
Task 3 completed
Task 4 completed
All tasks completed


The following example shows **parallelism with multiprocessing**:


In this example, we define the same task function as before. However, instead of creating threads, we make five processes using multiprocessing. Process class. Each process is assigned to execute the task function with a different name. The processes are started and then joined to wait for their completion.


In [13]:
import multiprocessing
import time

def task(name):
    print(f"Task {name} started")
    time.sleep(2)  # Simulating some time-consuming task
    print(f"Task {name} completed")

# Creating multiple processes
processes = []

for i in range(5):
    p = multiprocessing.Process(target=task, args=(i,))
    processes.append(p)
    p.start()

# Waiting for all processes to complete
for p in processes:
    p.join()

print("All tasks completed")

Task 0 started
Task 1 started
Task 2 started
Task 3 startedTask 4 started

Task 0 completed
Task 1 completed
Task 3 completedTask 4 completed

Task 2 completed
All tasks completed


When you run this code, you see that the tasks are executed in parallel.
Each process runs independently, utilizing separate CPU cores. As a result,
the tasks may be completed in any order, and youll observe a significant
reduction in the execution time compared to the multithreading example.

## Summary <a class="anchor" id="fourth-bullet"></a>

In this lession, we have seen how we can improve the performance of our programs using multiprocessing and multithreading.


Some of the **key-take-away points** are as follows:

1. Multithreading allows concurrent execution of multiple threads within a single process, improving responsiveness and enabling parallelism.

2. Understanding the Global Interpreter Lock (GIL) in Python is crucial when working with multithreading, as it restricts true parallelism for CPU-bound tasks.

3. Synchronization mechanisms like locks, semaphores, and condition variables ensure thread safety and avoid race conditions in multithreaded programs.

4. Multithreading is well-suited for I/O-bound tasks, where it can overlap I/O operations and maintain program responsiveness.

5. Debugging and troubleshooting multithreaded code requires careful consideration of synchronization issues, proper error handling, and utilizing logging and debugging tools.
